# Executive summary

1. Provide Contact tracing using apps people already trust. Users only need to opt-in, without installing anything.
1. Would reach most users quickly with buy-in from popular, trusted apps and providers (e.g. Maps and S3).
1. Propses a distributed solution, using cryptographically strong encryption.
  1. One-time pads stored on devices temporarily would be used as needed to identify hashes of recent locations.
  1. Users who were also at those locations at the same time could only be notified *passively*; in other words, it should be possible for them and only them to verify that there was a contact.
1. Proposes open source code and openly operated services to complement the eco-system, especially for people wanting more privacy.


# Proposal for a very private location tracker, for contact tracing

To prevent a virus from spreading, one approach is to notify people who have a high chance of being recently infected, for self-quarantine and testing. For every person who is newly confirmed as a carrier, their recent location history could be used to identify other people who have been in close proximity. However, most people do not want their location history public. This notebook is a proposal to share location in a cryptographically strong way, while making it likely that a high percentage of people who should be notified would be. They would be *passively* notified, their true identification being strongly encrypted.

The proposal would make both location and personal identification mostly private:
1. Trusted location and personal "authorities" could separately decrypt some of the information. (More than one authority could be used by each person for each type of information).
1. Full personal identification is possible only with encrypted information stored on device. 
1. The on-device store would destroy entries older than a set time, such that data from earlier than some number of days would be untraceable.
1. Requests for decryption could be further limited to trusted medical providers by agreement with PersonalAuthority's.

If a user already trusts a provider (e.g. map app) with their location data, the provider could act as an "on-device store". That way a user could opt in to this service by simply opting in from their existing trusted app(s). Note, if trusted provider acts as an on-device store, the user would be *actively* notified by the trusted provider.

This proposals require two publically available log files (on the cloud). Public, encrypted messages, including *passive* notification, are listed below like:

``` diff 
- red ``` 

On-device storage messages are formatted as:
``` diff
+ green ``` 

In addition, public keys for both types of authorities must be shared.
    
See nytimes story for background: https://www.nytimes.com/2020/03/19/us/coronavirus-location-tracking.html

# Non-private illustration

A pseudo code scatch of a non-privacy-aware modeule would help illustrate how people tracking could work. Privacy issues and solutions would follow.

In [1]:
# pseudo code to show basic notification workflow, using location and time

from collections import defaultdict

class NonPrivateTracker:
    def __init__(self, time_interval_to_keep='2 weeks'):
        self.location_log = defaultdict(list)
        self.person_to_spacetimes = defaultdict(list)
        self.spacetimes_to_persons = defaultdict(list)
        self.time_interval_to_keep = time_interval_to_keep
        
    def log(person, location, time):
        self.location_log[time].append((person, location))
        
    def process_log(time_window):
        # truncate entries older than time_interval_to_keep
        for time in time_window:
            for person, location in self.location_log[time]: 
                self.person_to_spacetimes[person].append((time, location))
                self.spacetimes_to_persons[(time, location)].append(person)

    def notify(new_carrier, time_window):
        spacetimes = self.person_to_spacetimes[new_carrier]
        to_notify = set()
        for log_time, location in spacetimes:
            if log_time in time_window:
                persons = self.spacetimes_to_persons[(log_time, location)]
                to_notify.add(persons)
        return to_notify - new_carrier

Essentially this tracker works as follows:
1. Maintain a log of append-only location entries, where each entry contain tuple `(time, location, person)`
1. Periodically the log would be proessed, to make futher queries fast, namely:
  1. O(num_location_entries_for_new_carrier) * O(num_colocated_people_with_new_carrier_in_specific_time_window)
1. If the log is peridically truncated, so that entries older than some time frame are removed, the run time estimate above would only depend on the second term, number of colocated people. If location granularity is relatively high, it seems reasonable that fewer than 100 people could share the same location at any time. That means the run time operation is constant time, O(1). Offline operation, log processing, can be done in parellel using map reduce.
1. Reported time would also have to be in some agreed-upon values, perhaps in 15 minute intervals starting from 00:00.
1. If a person was in multiple locations within the same time window, one entry per location should be added.

# Privacy issues includes:

1. Person identification is exposed - anyone can see where a person was at any time, for any person, given the log. This is not ideal for the person, as they may have gone to places they do not want to be public. (For example, a person may want their business address to not be publically associated with their personal identity).
2. Location identification is exposed - anyone can see had how many people each location had at any time, given the log. This is not ideal for the location owner, as they may not want traffic patterns to those locations to be known. (For example, a restaurant may want to keep number of customers private).

# Add Personal privacy

To solve for personal privacy, I propose the following:
1. Instead of logging identifiable person tokens in the tuple `(time, location, person)`, use a cryptically strong one time pad, for example: by `secrets.randbits(128)`.
1. One time pads should be stored on user device only, along with timestamp used
1. Periodically pads used earlier than some time interval must be purged
1. To limit who can read the matching one_time_pads, they could be encrypted using the public keys of one or more PersonalAuthority's.
1. The log entry (before location encryption) would be: `(time, location, encrypted_one_time_pad)`;<br/> On-device store would hold:

```diff
+ (time, \
+  one_time_pad, \
+  PersonalAuthority)
```

# Add Location privacy

To solve for location privacy, while allowing broadcasts of contaminated `(time, location)`s. I propose the following:
1. Instead of logging location tokens in the tuple `(time, location, person)`, encrypt location with the public keys of one or more LocationAuthority's
1. People already trust some organizations for their location history, for example, when using a GPS-based map application. 
1. The tracker app can only detect collocation when location is shared to at least one LocationAuthority's. As such, it should be enabled to share location 24 hours a day by default.
1. The tracker app may want to enable a user to block times and/or locations where they do not want to be tracked at all, for example, every night from 11pm to 6am. The user would then be responsible to notify people they often see in the untracked time-spaces.
1. The log entry would be:

```diff
- (time, \
-  encrypted_location, \
-  encrypted_one_time_pad)
```

Some futher precautions:
1. log entries from device should be submitted in batch, possibly from VPNs to hide originating location from network traces
1. log entries are to be published by PersonalAuthority's after aggregating and scrambling a large number of users and entries.

# How to find collocators, to notify passively

When a user is diagnosed, they could share their on-device store entries with the PersonalAuthority's. (This could be further restricted by trusted MedicalAuthority's only). 

The PersonalAuthority's would:
1. Find the `encrypted_one_time_pad` in the common logs for this user, by decrypting the encrypted_one_time_pad's with their private keys. Decrypted item must match on-device one_time_pad's. If they do not match, there are no gurantees those log entries originated from the user.
1. Update the log that the encrypted_location was contaminated at the associated time, by appending a log entry:

```diff
- (time, \
-  encrypted_location, \
-  timed_status_with_salt_encrypted_by_personal_authority, \
-  PersonalAuthorityID)
```

LocationAuthority's would scan the log for needed action:
1. Verify the relevant log entry was indeed from PersonalAuthority. (PersonalAuthority's public key should decrypt status_contaminated_encrypted_by_personal_authority successfully). 
1. Local Authority would then attempt to decrypt the encrypted_location. If decryption succeeds, the location was previously encrypted using the LocationAuthority's public key. As such, this particular LocationAuthority was responsible to decript the encrypted_location.
1. It should broadcast privately to other LocationAuthority's that there was a contamination event, at `(time, other_encrypted_location)`, by appending multiple log entries like: 

```diff
- (time, \
-  other_encrypted_location, \
-  timed_status_with_salt_encrypted_by_location_authority, \
-  LocationAuthorityID)
```
The contaminated location is encrypted multiple times, using each known LocationAuthority's public keys. 

LocationAuthority's would:
1. Treat the above broadcasted messages the same way as though a PersonalAuthority told them `(time, encrypted_location) `was contaminated, except no futher broadcasts was needed.
1. Decrypt location as needed, then along with passed in time field, look for other log entries encrypted using by its public key. 
1. Any colocated person would have a log entry keyed by `(time, encrypted_location)`. The `encryted_one_time_pad` used for that entry would then allow a user to know they could be contaminated.

To notify:
1. This would be done in a <b>separate log file</b>, keeping known entries like so:

```diff
- (time, encrypted_one_time_pad)
```
1. A user's location tracker app could check it's on-device storage for any matches, which would indicate its owner may be infected. The owner should then self-quanrantine and/or get tested.

# Considerations for nosy external agents

1. Police-state?
  1. If an external agent (e.g. police) want to track where a person went, the published log itself won't help, because both personal and locational information are encrypted. 
  1. Without access to an on-device store, but with access to the private keys of all the Location and Personal Authorities, an external agent could detect whether a location had at least one person at a time, and can estimate a plausible range of number of people there. However, the agent could not figure out who exactly, because the only available personal identification would be a single-use random number. 
  1. With access to the on-device store, and subpoenas at the Location and Personal Authorities, it is possible to track where a person went, provided the app was enabled in the time window of interest, and the time window was within the `time_interval_to_keep` as enforced in the on-device store. It is possible to know that some other people were co-located, but without their on-device store, it should be close to impossible to prove who those people were, using only data contained in this design. The main weak point would be network traces when Log entries were made.

1. Home church?
  1. If periodic meetings at the same location needs to stay secret, users should stop the tracking app from sharing their location around the time or space. Otherwise, the log lines could leak an increase of activity in an area. Note, this does not help against other tracking devices like cameras.

# Futher considerations

1. Denial of service on PersonAuthority or LocationAuthority? 
  1. Connection to Authorities could be protected by public-private key encryption. Messages have to be "sealed" with public keys of Authorities, and "signed" by Clients' private-keys.
  
1. How is Log maintained? If centralized, how to prevent DDOS? If distributed, how to verify content?
  1. Centralized providers such as AWS may be well suited for this. Entries could be bucketed by time and  encrypted_one_time_pad, with hashing to share capacity

# Adoption

1. Try to gain support from big players (Apple, Facebook, Google, etc) who already have location sensitive data. Most users are probably OK with these big players acting as their "device" for this service.
1. Publish protocol, open source code, and certify as [openly operated services](https://openlyoperated.org/how-to) to show viable alternatives to big players.
1. Opt-in using existing apps, or run new standalone app. Both should be set to be automatically dis-abled in a short amount of time, unless the user actively confirms a desire for further location tracking.